# Meta-Labeling Experiments: A Step-by-Step Guide

## Introduction

This notebook recreates the meta-labeling experiments from Hudson & Thames' research in a beginner-friendly manner. Meta-labeling is a machine learning technique that sits on top of a primary trading strategy to improve performance by filtering out false positive signals.

**Key Concepts:**
- **Primary Model**: Generates trading signals (buy/sell/hold)
- **Triple Barrier Method**: Advanced labeling technique that accounts for stop-loss, take-profit, and time-based exits
- **Meta-Labeling**: Secondary ML model that decides whether to act on primary model signals
- **Goal**: Improve Sharpe ratio, reduce drawdown, and increase precision

## Setup and Dependencies

In [1]:
import sys
sys.path.append("..")  # Adjust based on your directory structure

%reload_ext autoreload
%load_ext line_profiler
%autoreload 3 -p

# %aimport afml

In [ ]:
import warnings
import winsound
from datetime import timedelta, datetime as dt
from pprint import pprint
from pathlib import Path

import matplotlib.pyplot as plt
import MetaTrader5 as mt5
import seaborn as sns

from afml.data_structures.bars import *
from afml.labeling import (
    add_vertical_barrier,
    calculate_label_metrics,
    fixed_time_horizon,
    get_bins,
    get_bins_from_trend,
    get_events,
    triple_barrier_labels,
    trend_meta_labels,
)
from afml.strategies import (
    BaseStrategy,
    BollingerMeanReversionStrategy,
    MovingAverageCrossoverStrategy,
    TripleBarrierEvaluator,
    get_entries,
)
from afml.util import (
    COMMODITIES,
    CLEAN_DATA_PATH,
    CRYPTO,
    DATA_PATH,
    FX_MAJORS,
    PERCENTILES,
    UTC,
    DataFrameFormatter,
    get_ticks,
    load_tick_data,
    login_mt5,
    save_data_to_parquet,
    value_counts_data,
    verify_or_create_account_info,
)

warnings.filterwarnings("ignore")
plt.style.use("dark_background")


In [3]:
from afml.cache import clear_afml_cache


clear_afml_cache()

## 1. Data Preparation

In [4]:
account = login_mt5(account="FundedNext_STLR2_6K")
symbol = "EURUSD"
columns = ['bid', 'ask']
tick_bar_dict = {"M1": 50, "M5": 200, "M15": 700, "M30": 1000, "H1": 3000}
tick_df = None

start_date, end_date = "2018-01-01", "2024-12-31"
sample_start, sample_end = start_date, "2021-12-31"
oos_start = pd.Timestamp(sample_end) + timedelta(days=1) # Out-of-sample start date


def get_data(symbol, timeframe, dirpath="data"):
    """Returns time-bars and tick-bars for timeframe"""
    global tick_df
    timeframe = timeframe.title()
    directory = Path(dirpath)
    tick_bar_size = tick_bar_dict[timeframe]
    fname = Path(dirpath, f"{symbol}_{timeframe}_time_{start_date}-{end_date}.parq")
    fname1 = Path(dirpath, f"{symbol}_{timeframe}_tick-{tick_bar_size}_{start_date}-{end_date}.parq")
    time_bars, tick_bars = (None, None)

    # --- Construct the search pattern ---
    # The '*' is a wildcard that matches any sequence of characters
    search_pattern = f"{symbol}_{timeframe}_*"

    # --- Find the files ---
    # glob() returns a generator, so you can iterate over it
    found_files = directory.glob(search_pattern)

    # --- Print the results ---
    for file in found_files:
        if fname == file:
            time_bars = pd.read_parquet(fname)
            print("Loaded", file.name)
        elif fname1 == file:
            tick_bars = pd.read_parquet(fname1)
            print("Loaded", file.name)
        
    if any(x is None for x in (time_bars, tick_bars)):
        try:
            tick_df = load_tick_data(CLEAN_DATA_PATH, symbol, start_date, end_date, account, columns)
        except FileNotFoundError:
            save_data_to_parquet(CLEAN_DATA_PATH, symbol, start_date, end_date, account)
        if time_bars is None:
            time_bars = make_bars(tick_df, bar_type="time", timeframe=timeframe, price="bid_ask", verbose=True)
            time_bars.to_parquet(fname)
        if tick_bars is None:
            tick_bar_size = calculate_ticks_per_period(tick_df, timeframe)
            tick_bar_dict[timeframe] = tick_bar_size
            tick_bars = make_bars(tick_df, bar_type="tick", bar_size=tick_bar_size, price="bid_ask", verbose=True)
            fname1 = Path(dirpath, f"{symbol}_{timeframe}_tick-{tick_bar_size}_{start_date}-{end_date}.parq")
            tick_bars.to_parquet(fname1)

    return time_bars, tick_bars

2025-08-29 01:49:45.543 | INFO     | afml.util.get_data:login_mt5:82 - Attempting to log in to MT5 with account: FundedNext_STLR2_6K
2025-08-29 01:49:45.613 | SUCCESS  | afml.util.get_data:login_mt5:93 - Successfully logged in to MT5 as FundedNext_STLR2_6K.
2025-08-29 01:49:45.624 | INFO     | afml.util.get_data:login_mt5:95 - MT5 Version: (500, 5200, '1 Aug 2025')
2025-08-29 01:49:45.639 | INFO     | afml.util.get_data:login_mt5:98 - Connected to MetaTrader 5 at C:\Program Files\MetaTrader 5


## 2. Bollinger Band Strategy

In [5]:
timeframe = "M5"
tick_bar_size = tick_bar_dict[timeframe]
bb_time_bars, bb_tick_bars = get_data(symbol, timeframe, dirpath="data")
bar_size = f"tick-{bb_tick_bars.tick_volume.iloc[0]}"
bb_df = bb_time_bars.loc[sample_start : sample_end]
bb_df0 = bb_tick_bars.loc[sample_start : sample_end]

bb_period, bb_std = 20, 1.5 # Bollinger Band parameters
bb_strategy = BollingerMeanReversionStrategy(window=bb_period, num_std=bb_std)
target_vol_params = dict(days=1, lookback=100)
bb_strategy = BollingerMeanReversionStrategy(20, 1.5)
tb_evaluator = TripleBarrierEvaluator(
    bb_strategy, bb_df, target_vol_params,
    target_vol_multiplier=1,
    filter_events=False,
    vertical_barrier_zero=True,
    on_crossover=True,
    )
pt_barrier, sl_barrier, time_horizon = (1, 2, 20)

Loaded EURUSD_M5_tick-200_2018-01-01-2024-12-31.parq
Loaded EURUSD_M5_time_2018-01-01-2024-12-31.parq
2025-08-29 01:49:47.650 | INFO     | afml.strategies.genetic_optimizer:__init__:147 - Generating primary signals...
2025-08-29 01:49:47.768 | INFO     | afml.strategies.strategies:get_entries:76 - Generated 32,863 trade events generated by crossover.


### Fixed-Time Horizon Method

In [6]:
bb_events_time0 = tb_evaluator.evaluate_performance(0, 0, time_horizon)
bb_events_time_metrics0 = tb_evaluator.calculate_strategy_metrics(bb_events_time0)
bb_events_time_metrics0

total_return                         2.290849
annualized_return                      0.3477
volatility                           0.295733
downside_volatility                  0.219597
sharpe_ratio                         9.218397
sortino_ratio                         12.4145
var_95                              -0.001626
cvar_95                             -0.002557
skewness                             0.141904
kurtosis                             7.629223
probabilistic_sharpe_ratio                1.0
pos_concentration                    0.000075
neg_concentration                    0.000081
time_concentration                    0.00011
max_drawdown                         0.096605
avg_drawdown                         0.004586
drawdown_duration             2 days 03:33:46
ulcer_index                          0.010736
calmar_ratio                         3.599173
bet_frequency                           13366
bets_per_year                            3346
num_trades                        

### Triple-Barrier Method

In [7]:
tb_evaluator.set_params(strategy=bb_strategy, data=bb_df, filter_events=False, 
                        target_vol_params=target_vol_params,
                        vertical_barrier_zero=True)
bb_events_time = tb_evaluator.evaluate_performance(pt_barrier, sl_barrier, time_horizon)
bb_events_time_metrics = tb_evaluator.calculate_strategy_metrics(bb_events_time)
bb_events_time_metrics

2025-08-29 01:49:59.619 | INFO     | afml.strategies.genetic_optimizer:__init__:147 - Generating primary signals...
2025-08-29 01:49:59.703 | INFO     | afml.strategies.strategies:get_entries:76 - Generated 32,863 trade events generated by crossover.


total_return                         2.479918
annualized_return                    0.366693
volatility                           0.268633
downside_volatility                  0.207613
sharpe_ratio                        10.588295
sortino_ratio                       13.700293
var_95                              -0.001614
cvar_95                             -0.002423
skewness                             -0.44991
kurtosis                             4.252901
probabilistic_sharpe_ratio                1.0
pos_concentration                    0.000049
neg_concentration                    0.000074
time_concentration                    0.00011
max_drawdown                         0.091667
avg_drawdown                         0.004136
drawdown_duration             1 days 21:35:09
ulcer_index                          0.009637
calmar_ratio                         4.000261
bet_frequency                           13366
bets_per_year                            3346
num_trades                        

In [8]:
tb_evaluator.set_params(strategy=bb_strategy, data=bb_df, filter_events=True, 
                        target_vol_params=target_vol_params, vertical_barrier_zero=True)
bb_events_time_filt = tb_evaluator.evaluate_performance(pt_barrier, sl_barrier, time_horizon)
bb_events_time_filt_metrics = tb_evaluator.calculate_strategy_metrics(bb_events_time_filt)
bb_events_time_filt_metrics

2025-08-29 01:50:03.591 | INFO     | afml.strategies.genetic_optimizer:__init__:147 - Generating primary signals...
2025-08-29 01:50:08.738 | INFO     | afml.strategies.strategies:get_entries:76 - Generated 7,834 trade events generated by CUSUM filter.


total_return                         0.518471
annualized_return                    0.110312
volatility                           0.366952
downside_volatility                   0.27039
sharpe_ratio                         10.93901
sortino_ratio                        14.84557
var_95                               -0.00227
cvar_95                             -0.003161
skewness                            -0.446907
kurtosis                             2.334759
probabilistic_sharpe_ratio           0.999789
pos_concentration                    0.000156
neg_concentration                    0.000248
time_concentration                   0.003163
max_drawdown                         0.082104
avg_drawdown                           0.0049
drawdown_duration             6 days 07:32:24
ulcer_index                          0.010183
calmar_ratio                         1.343558
bet_frequency                            3541
bets_per_year                             886
num_trades                        

In [9]:
tb_evaluator.set_params(strategy=bb_strategy, data=bb_df0, filter_events=False, 
                        target_vol_params=target_vol_params, vertical_barrier_zero=True)
bb_events_tick = tb_evaluator.evaluate_performance(pt_barrier, sl_barrier, time_horizon)
bb_events_tick_metrics = tb_evaluator.calculate_strategy_metrics(bb_events_tick)
bb_events_tick_metrics

2025-08-29 01:50:15.364 | INFO     | afml.strategies.genetic_optimizer:__init__:147 - Generating primary signals...
2025-08-29 01:50:15.496 | INFO     | afml.strategies.strategies:get_entries:76 - Generated 45,846 trade events generated by crossover.


total_return                          1.99957
annualized_return                    0.316771
volatility                           0.283604
downside_volatility                  0.193847
sharpe_ratio                         9.182316
sortino_ratio                       13.433971
var_95                               -0.00142
cvar_95                             -0.001958
skewness                            -0.343923
kurtosis                             2.004854
probabilistic_sharpe_ratio                1.0
pos_concentration                    0.000023
neg_concentration                    0.000035
time_concentration                    0.00425
max_drawdown                         0.088499
avg_drawdown                         0.004424
drawdown_duration             1 days 23:57:29
ulcer_index                          0.010249
calmar_ratio                          3.57936
bet_frequency                           15594
bets_per_year                            3903
num_trades                        

In [10]:
tb_evaluator.set_params(strategy=bb_strategy, data=bb_df0, filter_events=True, 
                        target_vol_params=target_vol_params, vertical_barrier_zero=True)
bb_events_tick_filt = tb_evaluator.evaluate_performance(pt_barrier, sl_barrier, time_horizon)
bb_events_tick_filt_metrics = tb_evaluator.calculate_strategy_metrics(bb_events_tick_filt)
bb_events_tick_filt_metrics

2025-08-29 01:50:23.435 | INFO     | afml.strategies.genetic_optimizer:__init__:147 - Generating primary signals...
2025-08-29 01:50:23.554 | INFO     | afml.strategies.strategies:get_entries:76 - Generated 12,921 trade events generated by CUSUM filter.


total_return                         0.661697
annualized_return                    0.135668
volatility                           0.324231
downside_volatility                  0.226894
sharpe_ratio                        13.129728
sortino_ratio                       18.762367
var_95                              -0.001618
cvar_95                             -0.002275
skewness                            -0.382614
kurtosis                             1.896581
probabilistic_sharpe_ratio           0.999997
pos_concentration                    0.000084
neg_concentration                    0.000133
time_concentration                   0.006254
max_drawdown                         0.036675
avg_drawdown                         0.003499
drawdown_duration             3 days 21:51:15
ulcer_index                          0.006424
calmar_ratio                         3.699184
bet_frequency                            6145
bets_per_year                            1538
num_trades                        

In [11]:
print(f"Bollinger_w{bb_period}_std{bb_std}: \n")

bb_metrics_df = pd.DataFrame({
    "time": bb_events_time_metrics, 
    "filtered_time": bb_events_time_filt_metrics,
    "tick": bb_events_tick_metrics, 
    "filtered_tick": bb_events_tick_filt_metrics
    })
bb_metrics_df

Bollinger_w20_std1.5: 



,time,filtered_time,tick,filtered_tick
total_return,2.479918,0.518471,1.99957,0.661697
annualized_return,0.366693,0.110312,0.316771,0.135668
volatility,0.268633,0.366952,0.283604,0.324231
downside_volatility,0.207613,0.27039,0.193847,0.226894
sharpe_ratio,10.588295,10.93901,9.182316,13.129728
sortino_ratio,13.700293,14.84557,13.433971,18.762367
var_95,-0.001614,-0.00227,-0.00142,-0.001618
cvar_95,-0.002423,-0.003161,-0.001958,-0.002275
skewness,-0.44991,-0.446907,-0.343923,-0.382614
kurtosis,4.252901,2.334759,2.004854,1.896581


### Trend-Scanning Method

In [ ]:
span = (5, 21)
volatility_threshold = 0.1
volatility_threshold_mr = 0.85

side, t_events = get_entries(bb_strategy, bb_df)
close = bb_df.close

bb_trend_time = get_bins_from_trend(close, span, volatility_threshold)
bb_trend_time = trend_meta_labels(bb_trend_time, side, t_events)
bb_trend_time_metrics = calculate_label_metrics(bb_df.index, side, bb_trend_time)

bb_trend_time_mr = get_bins_from_trend(close, span, volatility_threshold_mr)
bb_trend_time_mr = trend_meta_labels(bb_trend_time_mr, side, t_events)
bb_trend_time_metrics_mr = calculate_label_metrics(bb_df.index, side, bb_trend_time_mr)

print(f"bb_trend_time.shape: {bb_trend_time.shape}, bb_trend_time_mr.shape: {bb_trend_time_mr.shape}\n")
print(f"Bollinger_w{bb_period}_std{bb_std}_trend_scanning: \n")
bb_trend_time_metrics_all = pd.concat([bb_trend_time_metrics, bb_trend_time_metrics_mr], axis=1)
bb_trend_time_metrics_all.columns = ["trend_time", "trend_time_mr"]
bb_trend_time_metrics_all

Reloading 'afml.labeling.trend_scanning'.
2025-08-29 02:02:11.435 | INFO     | afml.strategies.strategies:get_entries:76 - Generated 32,863 trade events generated by crossover.
bb_trend_time.shape: (25518, 8), bb_trend_time_mr.shape: (2415, 8)

Bollinger_w20_std1.5_trend_scanning: 



,trend_time,trend_time_mr
total_return,2.62614,0.225981
annualized_return,0.380552,0.052328
volatility,0.342942,0.558433
downside_volatility,0.234527,0.350261
sharpe_ratio,11.059245,11.453745
sortino_ratio,16.171624,18.261088
var_95,-0.001882,-0.003181
cvar_95,-0.002851,-0.00448
skewness,-0.030976,0.143253
kurtosis,5.096016,2.621596


## 3. Moving Average Crossover Strategy

In [ ]:
timeframe = "M30"
tick_bar_size = tick_bar_dict[timeframe]
ma_time_bars, ma_tick_bars = get_data(symbol, timeframe, dirpath="data")
bar_size = f"tick-{ma_tick_bars.tick_volume.iloc[0]}"
ma_df = ma_time_bars.loc[sample_start : sample_end]
ma_df0 = ma_tick_bars.loc[sample_start : sample_end]

fast_window, slow_window = 50, 200
ma_strategy = MovingAverageCrossoverStrategy(fast_window, slow_window)
target_vol_params = dict(days=1, lookback=100)
pt_barrier, sl_barrier, time_horizon = (3, 1, 200)
tb_evaluator = TripleBarrierEvaluator(
    ma_strategy, ma_df, target_vol_params,
    target_vol_multiplier=1,
    filter_events=False,
    vertical_barrier_zero=True,
    on_crossover=True,
    )

Loaded EURUSD_M30_tick-1000_2018-01-01-2024-12-31.parq
Loaded EURUSD_M30_time_2018-01-01-2024-12-31.parq
2025-08-29 00:44:40.736 | INFO     | afml.strategies.genetic_optimizer:__init__:146 - Generating primary signals...
2025-08-29 00:44:40.842 | INFO     | afml.strategies.strategies:get_entries:76 - Generated 287 trade events generated by crossover.


In [ ]:
tb_evaluator.set_params(strategy=ma_strategy, data=ma_df, filter_events=False, 
                        target_vol_params=target_vol_params)
ma_events_time = tb_evaluator.evaluate_performance(pt_barrier, sl_barrier, time_horizon)
ma_events_time_metrics = tb_evaluator.calculate_strategy_metrics(ma_events_time)
ma_events_time_metrics

2025-08-29 00:44:43.592 | INFO     | afml.strategies.genetic_optimizer:__init__:146 - Generating primary signals...
2025-08-29 00:44:43.613 | INFO     | afml.strategies.strategies:get_entries:76 - Generated 287 trade events generated by crossover.


total_return                           0.124133
annualized_return                      0.029747
volatility                             0.623553
downside_volatility                    0.139408
sharpe_ratio                           8.397366
sortino_ratio                         37.560363
var_95                                -0.005439
cvar_95                               -0.005955
skewness                               1.298916
kurtosis                               2.022453
probabilistic_sharpe_ratio             0.910468
pos_concentration                      0.003702
neg_concentration                      0.000791
time_concentration                     0.003894
max_drawdown                           0.048178
avg_drawdown                            0.01973
drawdown_duration             121 days 02:10:00
ulcer_index                            0.024585
calmar_ratio                           0.617439
bet_frequency                               288
bets_per_year                           

In [ ]:
tb_evaluator.set_params(strategy=ma_strategy, data=ma_df, filter_events=True, 
                        target_vol_params=target_vol_params)
ma_events_time_filt = tb_evaluator.evaluate_performance(pt_barrier, sl_barrier, time_horizon)
ma_events_time_filt_metrics = tb_evaluator.calculate_strategy_metrics(ma_events_time_filt)
ma_events_time_filt_metrics

2025-08-29 00:44:54.436 | INFO     | afml.strategies.genetic_optimizer:__init__:146 - Generating primary signals...
2025-08-29 00:44:55.004 | INFO     | afml.strategies.strategies:get_entries:76 - Generated 2,264 trade events generated by CUSUM filter.


total_return                           0.262574
annualized_return                      0.060148
volatility                              0.69363
downside_volatility                    0.185039
sharpe_ratio                            2.17703
sortino_ratio                          8.160748
var_95                                -0.006379
cvar_95                               -0.007652
skewness                                1.36425
kurtosis                               1.723162
probabilistic_sharpe_ratio             0.827655
pos_concentration                      0.000541
neg_concentration                      0.000139
time_concentration                     0.005818
max_drawdown                           0.272126
avg_drawdown                           0.117705
drawdown_duration             145 days 05:30:00
ulcer_index                            0.155526
calmar_ratio                           0.221028
bet_frequency                               273
bets_per_year                           

In [ ]:
tb_evaluator.set_params(strategy=ma_strategy, data=ma_df0, filter_events=False, 
                        target_vol_params=target_vol_params)
ma_events_tick = tb_evaluator.evaluate_performance(pt_barrier, sl_barrier, time_horizon)
ma_events_tick_metrics = tb_evaluator.calculate_strategy_metrics(ma_events_tick)
ma_events_tick_metrics

2025-08-29 00:44:56.561 | INFO     | afml.strategies.genetic_optimizer:__init__:146 - Generating primary signals...
2025-08-29 00:44:56.585 | INFO     | afml.strategies.strategies:get_entries:76 - Generated 516 trade events generated by crossover.


total_return                          -0.015796
annualized_return                     -0.003981
volatility                             0.627777
downside_volatility                    0.140874
sharpe_ratio                          -0.738385
sortino_ratio                         -3.290468
var_95                                -0.004367
cvar_95                               -0.005046
skewness                               0.937775
kurtosis                              -0.460891
probabilistic_sharpe_ratio             0.454487
pos_concentration                       0.00124
neg_concentration                      0.000338
time_concentration                     0.005865
max_drawdown                           0.069966
avg_drawdown                           0.016848
drawdown_duration             121 days 08:01:50
ulcer_index                            0.028707
calmar_ratio                          -0.056897
bet_frequency                               517
bets_per_year                           

In [ ]:
tb_evaluator.set_params(strategy=ma_strategy, data=ma_df0, filter_events=True, 
                        target_vol_params=target_vol_params)
ma_events_tick_filt = tb_evaluator.evaluate_performance(pt_barrier, sl_barrier, time_horizon)
ma_events_tick_filt_metrics = tb_evaluator.calculate_strategy_metrics(ma_events_tick_filt)
ma_events_tick_filt_metrics

2025-08-29 00:44:58.406 | INFO     | afml.strategies.genetic_optimizer:__init__:146 - Generating primary signals...
2025-08-29 00:44:58.444 | INFO     | afml.strategies.strategies:get_entries:76 - Generated 3,446 trade events generated by CUSUM filter.


total_return                          0.220331
annualized_return                     0.051148
volatility                            0.674952
downside_volatility                   0.172898
sharpe_ratio                          2.169734
sortino_ratio                         8.470124
var_95                               -0.004621
cvar_95                              -0.005546
skewness                              1.072288
kurtosis                              0.251129
probabilistic_sharpe_ratio            0.809826
pos_concentration                     0.000252
neg_concentration                     0.000075
time_concentration                     0.00977
max_drawdown                          0.249809
avg_drawdown                          0.079325
drawdown_duration             85 days 15:49:44
ulcer_index                           0.117637
calmar_ratio                          0.204748
bet_frequency                              501
bets_per_year                              125
num_trades   

In [ ]:
print(f"MACrossover_fast{fast_window}_slow{slow_window}: \n")

ma_metrics_df = pd.DataFrame({
    "time": ma_events_time_metrics, 
    "filtered_time": ma_events_time_filt_metrics,
    "tick": ma_events_tick_metrics, 
    "filtered_tick": ma_events_tick_filt_metrics
    })
ma_metrics_df

MACrossover_fast50_slow200: 



,time,filtered_time,tick,filtered_tick
total_return,0.124133,0.262574,-0.015796,0.220331
annualized_return,0.029747,0.060148,-0.003981,0.051148
volatility,0.623553,0.69363,0.627777,0.674952
downside_volatility,0.139408,0.185039,0.140874,0.172898
sharpe_ratio,8.397366,2.17703,-0.738385,2.169734
sortino_ratio,37.560363,8.160748,-3.290468,8.470124
var_95,-0.005439,-0.006379,-0.004367,-0.004621
cvar_95,-0.005955,-0.007652,-0.005046,-0.005546
skewness,1.298916,1.36425,0.937775,1.072288
kurtosis,2.022453,1.723162,-0.460891,0.251129
